![image.png](attachment:image.png)

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [ ]:
!pip install pyspark


     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 51.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=94590fc885a3a3c0d94f0b2913b031e66d3933b390564964b306a284e19d7245
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:

import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession.builder.getOrCreate()

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [ ]:
df = spark.read.format('csv')\
.option("delimiter", "\t")\
.option("inferSchema", "true")\
.load("SMSSpamCollection")\
.coalesce(5)

### Print the schema

In [ ]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



### Rename the first column to 'class' and second column to 'text'

In [ ]:
 df = df.selectExpr("_c0 as class", "_c1 as text")

In [ ]:
df.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [ ]:
df.show(n=10,truncate=True)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows



In [ ]:
df.show(n=10,truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [ ]:
df=df.withColumn('lenOtext',F.length('text'))

In [ ]:
df.show(1)

+-----+--------------------+--------+
|class|                text|lenOtext|
+-----+--------------------+--------+
|  ham|Go until jurong p...|     111|
+-----+--------------------+--------+
only showing top 1 row



### Show the new dataframe

In [ ]:
df.show()

+-----+--------------------+--------+
|class|                text|lenOtext|
+-----+--------------------+--------+
|  ham|Go until jurong p...|     111|
|  ham|Ok lar... Joking ...|      29|
| spam|Free entry in 2 a...|     155|
|  ham|U dun say so earl...|      49|
|  ham|Nah I don't think...|      61|
| spam|FreeMsg Hey there...|     147|
|  ham|Even my brother i...|      77|
|  ham|As per your reque...|     160|
| spam|WINNER!! As a val...|     157|
| spam|Had your mobile 1...|     154|
|  ham|I'm gonna be home...|     109|
| spam|SIX chances to wi...|     136|
| spam|URGENT! You have ...|     155|
|  ham|I've been searchi...|     196|
|  ham|I HAVE A DATE ON ...|      35|
| spam|XXXMobileMovieClu...|     149|
|  ham|Oh k...i'm watchi...|      26|
|  ham|Eh u remember how...|      81|
|  ham|Fine if thats th...|      56|
| spam|England v Macedon...|     155|
+-----+--------------------+--------+
only showing top 20 rows



### Get the average text length for each class (give alias name to the average length column)

In [ ]:
df.groupBy('class').agg(F.avg('lenOtext').alias('AVG.LENGTH')).show()

+-----+-----------------+
|class|       AVG.LENGTH|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [ ]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_out")
stopwords = StopWordsRemover(inputCol='token_out',outputCol='stop_out')
count_vec = CountVectorizer(inputCol='stop_out',outputCol='count_vector')
idf = IDF(inputCol="count_vector", outputCol="tf_idf")

- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [ ]:
stringConv = StringIndexer(inputCol='class',outputCol='label')

In [ ]:
from pyspark.ml.feature import VectorAssembler
features=VectorAssembler(inputCols=['tf_idf','lenOtext'],outputCol='features')

## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [ ]:
from pyspark.ml.classification import NaiveBayes


In [ ]:
classifier=NaiveBayes()

## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [ ]:
from pyspark.ml.pipeline import Pipeline

In [ ]:
pipe=Pipeline(stages=[stringConv,tokenizer,stopwords,count_vec,idf,features,classifier])

### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [ ]:
seed = 42
splits = df.randomSplit([0.7, 0.3], seed)
train_df = splits[0]
test_df = splits[1]

### Fit your Pipeline model to the training data

In [ ]:
pipeline_model=pipe.fit(train_df)

### Perform predictions on tests dataframe

In [ ]:
Test1=pipeline_model.transform(test_df)

### Print the schema of the prediction dataframe

In [ ]:
Test1.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- lenOtext: integer (nullable = true)
 |-- label: double (nullable = false)
 |-- token_out: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_out: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- count_vector: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
eval=MulticlassClassificationEvaluator()
score1=eval.evaluate(Test1)

In [ ]:
score1

0.9727502290227267

# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>